In [1]:
from injector_surrogate_quads import Surrogate_NN
import torch
import torch.nn as nn

import sys,os
sys.path.append('../configs')
#Sim reference point to optimize around
from ref_config import ref_point

In [2]:
from tensorflow.keras.layers import InputLayer, Dense, Dropout

Model = Surrogate_NN()

Model.load_saved_model(model_path = '../models/', \
                       model_name = 'model_OTR2_NA_rms_emit_elu_2021-07-27T19_54_57-07_00')
Model.load_scaling()
Model.take_log_out = False

# print(Model.model_1.summary())

class Surrogate_NN_PyTorch(nn.Module):
    def __init__(self, model=None):
        super(Surrogate_NN_PyTorch, self).__init__()
        
        self.modules = []

        if model != None: 
            for layer in model.model_1.layers:  
                config = layer.get_config()
                weights = layer.get_weights()

                if (isinstance(layer,Dense)):
                    w = torch.as_tensor(weights[0])
                    b = torch.as_tensor(weights[1])

                    linear = nn.Linear(w.shape[0], 
                                        w.shape[1], 
                                        bias=config["use_bias"])

                    with torch.no_grad():
                        linear.weight = torch.nn.Parameter(w.transpose(0,1))
                        linear.bias = torch.nn.Parameter(b)

                    self.modules.append(linear) 

                    # activation function
                    if (config["activation"]=='elu'):
                        self.modules.append(nn.ELU())

                elif (isinstance(layer,Dropout)):
                    self.modules.append(nn.Dropout(p=config["rate"]))

                elif (isinstance(layer,InputLayer) == False):
                    print("error")
        
        self.network = nn.Sequential(*self.modules)
    
    def forward(self, x):
        y = self.network(x)
        return y

test_model = Surrogate_NN_PyTorch(Model)
test_model.eval()
torch.save(test_model, '../models/Surrogate_NN_PyTorch.pth')

2022-07-09 23:47:13.085082: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
class MinMaxScaler_PyTorch(object):
    def __init__(self, transformer):
        super(MinMaxScaler_PyTorch, self).__init__()
        self.scale = torch.as_tensor(transformer.scale_)
        self.min = torch.as_tensor(transformer.min_)

    def transform(self, x): 
        x *= self.scale
        x += self.min
        return x
    
    def inverse_transform(self, x):
        x -= self.min
        x /= self.scale
        return x

In [4]:
import numpy as np
n_samples = 3
n_var = 9

test_transformer_x = MinMaxScaler_PyTorch(Model.transformer_x)
test_transformer_y = MinMaxScaler_PyTorch(Model.transformer_y)

torch.save(test_transformer_x, '../data/transformer_x_pytorch.pth')
torch.save(test_transformer_y, '../data/transformer_y_pytorch.pth')

keras_test_point = Model.sim_to_machine(np.asarray(ref_point))
torch_test_point = torch.as_tensor(Model.sim_to_machine(ref_point))
# print(keras_test_point)
# print(torch_test_point)

# original surrogate 
keras = Model.pred_machine_units(keras_test_point)
print(keras)

# PyTorch surrogate
# print(Model.transformer_x.transform(test_point))

col = []
scale = []
for i in range(0,len(Model.model_in_list)):
    col.append(Model.loc_in[Model.model_in_list[i]])
    scale.append(Model.pv_to_sim_factor[Model.sim_name_to_pv_name[Model.model_in_list[i]]])

torch_test_point[:,col] =  torch_test_point[:,col] * torch.tensor(scale)
normed_x = test_transformer_x.transform(torch_test_point)

test_model.double()
pytorch = test_model(normed_x).detach()

# print(ha)
print(test_transformer_y.inverse_transform(pytorch.detach()))

[[1.2124054e-04 1.1827494e-04 4.6100092e-04 5.5473919e-07 5.6714555e-07]]
tensor([[1.2124e-04, 1.1827e-04, 4.6100e-04, 5.5474e-07, 5.6715e-07]],
       dtype=torch.float64)


2022-07-09 23:47:13.325126: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-07-09 23:47:13.327434: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1996125000 Hz
